In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import LineString
from utils import convert_3d_to_2d, create_transect, split_polygon_with_polylines, percentile

points_full_res_path = "C:/Users/eleonore.kong/Documents/NGE/170D_Uruguay_TramoA_fullres/170D_Uruguay_TramoA_fullres.shp"
points_rural_path = "C:/Users/eleonore.kong/Documents/NGE/170D_Uruguay_TramoA_rural/170D_Uruguay_TramoA_rural.shp"
trace_path = "C:/Users/eleonore.kong/Documents/NGE/Trace.geojson"
trace_id = "'TRAMO 1'"
output_folder = "C:/Users/eleonore.kong/Documents/NGE/"


gdf_trace = gpd.read_file(trace_path, where=f"Layer LIKE {trace_id}")
crs = gdf_trace.crs
gdf_trace['geometry'] = gdf_trace['geometry'].apply(convert_3d_to_2d)
gdf_trace['length'] = gdf_trace['geometry'].length
gdf_trace = gdf_trace[gdf_trace['length'] >= 1000]
# trace_polyline = unary_union(gdf_trace.geometry)

points_full_res = gpd.read_file(points_full_res_path)
points_rural = gpd.read_file(points_rural_path)
points_all = pd.concat([points_full_res, points_rural], ignore_index=True)
points_all = points_all.to_crs(crs)
points_all['abs_velocity'] = np.abs(points_all['velocity'])
# points_all = points_full_res.to_crs(3857)

In [3]:
buffer_dict = {5000: 1, 1000: 2, 500: 3, 200: 4, 100: 4}

for buffer_width, ratio in buffer_dict.items():
    stats_gdfs = []
    for trace_polyline in gdf_trace['geometry']:
        trace_polyline = trace_polyline.simplify(tolerance=50)
        transect_step = int(buffer_width * ratio)
        num_points = int(trace_polyline.length // transect_step)
        num_points += 1
        points_along_trace = [trace_polyline.interpolate(transect_step * i) for i in range(num_points + 1)]
        # points_along_trace_gdf = gpd.GeoDataFrame(geometry=points_along_trace, crs=gdf_trace.crs)
        line = LineString(points_along_trace)
    
        buffer = trace_polyline.buffer(buffer_width, cap_style="round", join_style="bevel")
    
        num_transects = int(line.length / transect_step)
        transect_length = buffer_width + buffer_width * 2.5
        transects = [create_transect(line, transect_step * i, transect_length) for i in range(0, num_transects + 1)]
        transects_gdf = gpd.GeoDataFrame(geometry=transects, crs=crs)
        # transects_output = output_folder + f"insar_transect_line_{buffer_width}m.geojson"
        # transects_gdf.to_file(transects_output, driver="GeoJSON")
    
        split_polygons = split_polygon_with_polylines(buffer, transects)
        split_polygons_gdf = gpd.GeoDataFrame(geometry=split_polygons, crs=crs)
    
        join = points_all.sjoin(split_polygons_gdf)
        agg_points = join.groupby("index_right").agg({
            "velocity": [
                "size",
                "mean",
                "median",
                percentile(80),
                percentile(20)
            ],
            "abs_velocity": [
                "size",
                "mean",
                "median",
                percentile(80),
                percentile(20)
            ]
        }
        )
    
        agg_points.columns = ['_'.join(col) for col in agg_points.columns]
        stats_gdf = split_polygons_gdf.reset_index().merge(agg_points, right_on='index_right', left_on="index")
        stats_gdf.set_index("index", inplace=True)
        stats_gdfs.append(stats_gdf)
    
    gdf = pd.concat(stats_gdfs)
    gdf = gpd.GeoDataFrame(gdf, crs=crs)
    output_file = output_folder + f"insar_transect_stats_{buffer_width}m.geojson"
    gdf.to_file(output_file, driver="GeoJSON")
